In [2]:
import os
import pandas as pd
import numpy as np
import gensim
from gensim import corpora, models, similarities
import nltk


C:\Users\Vojta-Acer\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [5]:
# Load word Embedings
#DataPath = 'C:\\Users\\Vojta-Acer\\Desktop\\Diplomka\\word2vec\\glove.twitter.27B.200d_edited.txt'
#GLOVEmodel = gensim.models.KeyedVectors.load_word2vec_format(DataPath)

DataPath = 'C:\\Users\\Vojta-Acer\\Desktop\\Diplomka\\word2vec\\GoogleNews-vectors-negative300.bin'
W2Vmodel = gensim.models.KeyedVectors.load_word2vec_format(DataPath, binary=True)

In [8]:
def getTFIDF(documents):
    #calculate tfidf
    tf = documents.apply(pd.value_counts).fillna(0)
    idf = np.log((len(documents) + 1 ) / (tf.gt(0).sum() + 1))
    tfidf = tf * idf
    return tf, idf, tfidf

def Tweet2Vec_averaging(tokens):
    tweetVec = []
    for word in tokens:
        try:
            wordVec = W2Vmodel.wv[word]
            tweetVec.append(wordVec)
        except: continue            
    return np.mean(tweetVec, axis=0)

def Tweet2Vec_min(tokens):
    tweetVec = []
    for word in tokens:
        try:        
            wordVec = W2Vmodel.wv[word]
            tweetVec.append(wordVec)
        except: continue
    return np.min(tweetVec, axis=0)

def Tweet2Vec_max(tokens):
    tweetVec = []
    for word in tokens:
        try:
            wordVec = W2Vmodel.wv[word]
            tweetVec.append(wordVec)
        except: continue
    return np.max(tweetVec, axis=0)

def Tweet2Vec_minmax(tokens):
    tweetVec = []
    for word in tokens:
        try:        
            wordVec = W2Vmodel.wv[word]
            tweetVec.append(wordVec)
        except: continue

    minVec = np.min(tweetVec, axis=0)
    maxVec = np.max(tweetVec, axis=0)
    return np.append(maxVec, minVec)

def Tweet2Vec_tfidf(tokens):
    tweetVec = []
    for word in tokens:
        try:        
            wordVec = W2Vmodel.wv[word]
            weight = idfDictionary[word]
            tweetVec.append(wordVec*weight)
        except: continue            
    return np.mean(tweetVec, axis=0)


In [13]:
#test sample
test = ['no', 'yes']
test1 = ['test', 'test', 'also', 'suggest', 'work', 'provide', 'pavan']
test2 = ['boost', 'bla', 'use', 'test', 'be', 'provide']
test3 = ['test', 'blood', 'go', 'honor', 'test', 'provide', 'test']
test4 = ['boost', 'use', 'use', 'use', 'provide']

#test dataframe
df = pd.DataFrame()
df['new'] = [test , test1, test2, test3, test4]

# create tfidf weighting for each word
tf, idf, tfidf2 = getTFIDF(df['new'])
idfDictionary = idf.to_dict()


In [11]:
df['vec'] = df['new'].apply(Tweet2Vec_averaging)
twitterVec = df['vec'].apply(pd.Series)
twitterVec

0         1         2         3         4         5         6    \
0  0.099121 -0.140625  0.065918  0.097900 -0.085144  0.056396  0.112305   
1 -0.094012 -0.038932 -0.013380  0.010585  0.018188  0.084359  0.048305   
2 -0.103678 -0.029500  0.018636  0.039022 -0.081055  0.058879 -0.022461   
3 -0.146310 -0.006348  0.103324  0.043562  0.070330  0.016357  0.038435   
4  0.018359 -0.088086  0.004688  0.038135 -0.121973  0.089648  0.044629   

        7         8         9      ...          290       291       292  \
0 -0.151855  0.168945  0.186523    ...    -0.117401  0.071411  0.014923   
1 -0.053467  0.030648  0.115688    ...    -0.006304  0.077924  0.019252   
2 -0.094157  0.015767  0.143840    ...     0.045176  0.104370  0.007271   
3 -0.114607  0.041504  0.168128    ...     0.012530  0.121024 -0.125977   
4 -0.103613  0.016772  0.142725    ...     0.011877  0.113574  0.027100   

        293       294       295       296       297       298       299  
0  0.006958 -0.067261 -0.034729 -0.015686 -0.146973 -0.015625 -0.131958  
1  0.079102 -0.047991  0.030692 -0.013079 -0.022014  0.039969 -0.139768  
2 -0.016418 -0.055298  0.001912 -0.064616 -0.108042 -0.022949 -0.042155  
3  0.099548 -0.104762  0.064697 -0.018345  0.006304 -0.021678 -0.140346  
4 -0.049927 -0.083545 -0.045947  0.071777 -0.127734 -0.032812 -0.024121  

[5 rows x 300 columns]

In [35]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(preprocessor=lambda x: x, tokenizer=lambda x: x,  norm="l2")
tfidf_matrix = tfidf.fit_transform( df['new'])

In [36]:
pd.DataFrame(tfidf_matrix.toarray())

0         1         2         3         4         5         6   \
0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
1  0.404509  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2  0.000000  0.495819  0.495819  0.000000  0.400024  0.000000  0.000000   
3  0.000000  0.000000  0.000000  0.368755  0.000000  0.368755  0.368755   
4  0.000000  0.000000  0.000000  0.000000  0.308789  0.000000  0.000000   

         7         8         9         10        11        12        13  \
0  0.707107  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
1  0.000000  0.404509  0.227893  0.404509  0.541808  0.000000  0.404509   
2  0.000000  0.000000  0.279336  0.000000  0.332056  0.400024  0.000000   
3  0.000000  0.000000  0.207750  0.000000  0.740878  0.000000  0.000000   
4  0.000000  0.000000  0.215627  0.000000  0.000000  0.926366  0.000000   

         14  
0  0.707107  
1  0.000000  
2  0.000000  
3  0.000000  
4  0.000000

In [31]:
tfidf2

also        be       bla     blood     boost        go     honor  \
0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
1  1.098612  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2  0.000000  1.098612  1.098612  0.000000  0.693147  0.000000  0.000000   
3  0.000000  0.000000  0.000000  1.098612  0.000000  1.098612  1.098612   
4  0.000000  0.000000  0.000000  0.000000  0.693147  0.000000  0.000000   

         no     pavan   provide   suggest      test       use      work  \
0  1.098612  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
1  0.000000  1.098612  0.182322  1.098612  0.810930  0.000000  1.098612   
2  0.000000  0.000000  0.182322  0.000000  0.405465  0.693147  0.000000   
3  0.000000  0.000000  0.182322  0.000000  1.216395  0.000000  0.000000   
4  0.000000  0.000000  0.182322  0.000000  0.000000  2.079442  0.000000   

        yes  
0  1.098612  
1  0.000000  
2  0.000000  
3  0.000000  
4  0.000000